In [2]:
%%capture
%pip install -U transformers
%pip install -U datasets
%pip install -U accelerate
%pip install -U peft
%pip install -U trl
%pip install -U bitsandbytes
%pip install huggingface_hub[hf_xet]

In [3]:
from huggingface_hub import login
import os

hf_token = os.environ.get("HF_TOKEN")
login(hf_token)

In [4]:
import torch
torch.cuda.is_available()

True

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [7]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [8]:
model_dir = "Qwen/Qwen3-0.6B"

tokenizer = AutoTokenizer.from_pretrained(model_dir, use_fast=True)

model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    # quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)

model.config.use_cache = False
model.config.pretraining_tp = 1

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.50G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [9]:
!nvidia-smi

Fri Jun 27 11:14:17 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   54C    P0             27W /   70W |    1556MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [10]:
train_prompt_style = """You are a multilingual translation evaluation expert. Your task is to predict the quality score for translation pairs.
The quality score is a number between -2.0 and 2.0 and could fall into the following categories:
Scoring Criteria:
-2.0 to -1.0: Poor or very poor translation with meaning deviation or severe errors.
-1.0 to 0.0: Flawed translation but understandable.
0.0 to 1.0: Good translation in general.
1.0 to 2.0: Excellent or flawless translation.
The relative position of the scores in this range indicates subtle differences in translation quality.

### Source sentence:
{src}

### MT sentence:
{mt}

### Quality score:
{score}
"""

In [11]:
EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(df_example):
    srcs    = df_example["src"]
    mts     = df_example["mt"]
    scores  = df_example["zmean"]
    texts   = []

    for src, mt, score in zip(srcs, mts, scores):
        # 保证score为float并格式化到4位小数（或你自己希望的位数）
        score_str = f"{float(score):.4f}"
        # 格式化prompt
        text = train_prompt_style.format(src=src, mt=mt, score=score_str)
        texts.append(text)

    return {"text": texts}

In [12]:
trainpath = "/content/drive/MyDrive/qwen_mtqe_project/data/toy_train.tsv"
devpath = "/content/drive/MyDrive/qwen_mtqe_project/data/toy_dev.tsv"
testpath = "/content/drive/MyDrive/qwen_mtqe_project/data/toy_test.tsv"

In [13]:
import pandas as pd
from datasets import Dataset
traindf = pd.read_csv(trainpath, sep="\t")
devdf = pd.read_csv(devpath, sep="\t")
testdf = pd.read_csv(testpath, sep="\t")

train_dataset = Dataset.from_pandas(traindf)
dev_dataset = Dataset.from_pandas(devdf)
test_dataset = Dataset.from_pandas(testdf)

train_dataset = train_dataset.map(
    formatting_prompts_func,
    batched=True,
)
dev_dataset = dev_dataset.map(
    formatting_prompts_func,
    batched=True,
)
test_dataset = test_dataset.map(
    formatting_prompts_func,
    batched=True,
)

print(train_dataset["text"][16])

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/125 [00:00<?, ? examples/s]

Map:   0%|          | 0/125 [00:00<?, ? examples/s]

You are a multilingual translation evaluation expert. Your task is to predict the quality score for translation pairs.
The quality score is a number between -2.0 and 2.0 and could fall into the following categories:
Scoring Criteria:
-2.0 to -1.0: Poor or very poor translation with meaning deviation or severe errors.
-1.0 to 0.0: Flawed translation but understandable.
0.0 to 1.0: Good translation in general.
1.0 to 2.0: Excellent or flawless translation.
The relative position of the scores in this range indicates subtle differences in translation quality.

### Source sentence:
Confronting Scalpel for his misdeeds, she tortures him with his own scalpel, and leaves him disfigured and near death.

### MT sentence:
面对福音的恶行 ， 她用他自己的手术刀折磨他 ， 使他毁容 ， 差点就死了。

### Quality score:
-0.4941



In [14]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

Model Inference before Finetuning

In [15]:
inference_prompt_style = """You are a multilingual translation evaluation expert. Your task is to predict the quality score for translation pairs.
The quality score is an exact number between -2.0 and 2.0 and could fall into the following categories:
Scoring Criteria:
-2.0 to -1.0: Poor or very poor translation with meaning deviation or severe errors.
-1.0 to 0.0: Flawed translation but understandable.
0.0 to 1.0: Good translation in general.
1.0 to 2.0: Excellent or flawless translation.
The relative position of the scores in this range indicates subtle differences in translation quality.

### Source sentence:
{src}

### MT sentence:
{mt}

### Your score:
{score}"""

In [16]:
src_sentence = train_dataset['src'][16]
mt_sentence = train_dataset['mt'][16]
inputs = tokenizer(
    [inference_prompt_style.format(src=src_sentence, mt=mt_sentence, score="") + tokenizer.eos_token],
    return_tensors="pt"
).to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    eos_token_id=tokenizer.eos_token_id,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(response)

['You are a multilingual translation evaluation expert. Your task is to predict the quality score for translation pairs.\nThe quality score is an exact number between -2.0 and 2.0 and could fall into the following categories:\nScoring Criteria:\n-2.0 to -1.0: Poor or very poor translation with meaning deviation or severe errors.\n-1.0 to 0.0: Flawed translation but understandable.\n0.0 to 1.0: Good translation in general.\n1.0 to 2.0: Excellent or flawless translation.\nThe relative position of the scores in this range indicates subtle differences in translation quality.\n\n### Source sentence:\nConfronting Scalpel for his misdeeds, she tortures him with his own scalpel, and leaves him disfigured and near death.\n\n### MT sentence:\n面对福音的恶行 ， 她用他自己的手术刀折磨他 ， 使他毁容 ， 差点就死了。\n\n### Your score:\n\n\n\n</think>\n\n-1.0']


In [23]:
import re
def extract_score_from_response(text, default_score=-2.0):
    # 定位 "### Your score:" 后最近的一个浮点数
    pattern = r"### Your score:\s*.*?(-?\d+\.\d+|-?\d+)"
    match = re.search(pattern, text, re.DOTALL)
    if match:
        return float(match.group(1))
    else:
        return default_score
print(type(response[0]))
print(response[0])
score = extract_score_from_response(response[0])
print(score)

<class 'str'>
You are a multilingual translation evaluation expert. Your task is to predict the quality score for translation pairs.
The quality score is an exact number between -2.0 and 2.0 and could fall into the following categories:
Scoring Criteria:
-2.0 to -1.0: Poor or very poor translation with meaning deviation or severe errors.
-1.0 to 0.0: Flawed translation but understandable.
0.0 to 1.0: Good translation in general.
1.0 to 2.0: Excellent or flawless translation.
The relative position of the scores in this range indicates subtle differences in translation quality.

### Source sentence:
यस धाराका नाटकहरूमा स्वैरकल्पनाकाे प्रयाेग पनि पाइन्छ ।

### MT sentence:
It is also found in the dances of this patriarch, the prégé of alternatives.

### Your score:
ිවේ

-2.0


In [18]:
from trl import SFTTrainer
from transformers import TrainingArguments

train_dataset = train_dataset.remove_columns([col for col in train_dataset.column_names if col != "text"])

# Training Arguments
training_arguments = TrainingArguments(
    output_dir="output",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    num_train_epochs=2,
    logging_steps=0.2,
    warmup_steps=10,
    logging_strategy="steps",
    learning_rate=2e-4,
    fp16=False,
    bf16=False,
    group_by_length=True,
    report_to="none"
)

# Initialize the Trainer
trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=train_dataset,
    data_collator=data_collator,
)

Adding EOS to train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [19]:
import gc, torch
gc.collect()
torch.cuda.empty_cache()
model.config.use_cache = False
trainer.train()

Step,Training Loss
200,1.466700
400,1.358000
600,1.032200
800,0.776800
1000,0.728100


TrainOutput(global_step=1000, training_loss=1.0723645782470703, metrics={'train_runtime': 1413.6564, 'train_samples_per_second': 1.415, 'train_steps_per_second': 0.707, 'total_flos': 1355499834703872.0, 'train_loss': 1.0723645782470703})

In [21]:
src_sentence = traindf['src'][16]
mt_sentence = traindf['mt'][16]
inputs = tokenizer(
    [inference_prompt_style.format(src=src_sentence, mt=mt_sentence, score="") + tokenizer.eos_token],
    return_tensors="pt"
).to("cuda")
outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    eos_token_id=tokenizer.eos_token_id,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs, skip_special_tokens=True)
score = extract_score_from_response(response[0])
print(score)
print(response[0].split("### Response:"))

-1.4782
['You are a multilingual translation evaluation expert. Your task is to predict the quality score for translation pairs.\nThe quality score is an exact number between -2.0 and 2.0 and could fall into the following categories:\nScoring Criteria:\n-2.0 to -1.0: Poor or very poor translation with meaning deviation or severe errors.\n-1.0 to 0.0: Flawed translation but understandable.\n0.0 to 1.0: Good translation in general.\n1.0 to 2.0: Excellent or flawless translation.\nThe relative position of the scores in this range indicates subtle differences in translation quality.\n\n### Source sentence:\nConfronting Scalpel for his misdeeds, she tortures him with his own scalpel, and leaves him disfigured and near death.\n\n### MT sentence:\n面对福音的恶行 ， 她用他自己的手术刀折磨他 ， 使他毁容 ， 差点就死了。\n\n### Your score:\nාදී මෙම පිහිස් මත ප්\u200dරත්\u200dයයේ පෙස දෙස දෙස වැඩි ප්\u200dරත්\u200dයයේ පිහුවය විය.\n\n### Quality score:\n-1.4782\n']


In [ ]:
# print(dev_dataset['zmean'][0])
# src_sentence = dev_dataset['src'][0]
# mt_sentence = dev_dataset['mt'][0]
# inputs = tokenizer(
#     [inference_prompt_style.format(src=src_sentence, mt=mt_sentence, score="") + tokenizer.eos_token],
#     return_tensors="pt"
# ).to("cuda")
# outputs = model.generate(
#     input_ids=inputs.input_ids,
#     attention_mask=inputs.attention_mask,
#     max_new_tokens=1200,
#     eos_token_id=tokenizer.eos_token_id,
#     use_cache=True,
# )
# response = tokenizer.batch_decode(outputs, skip_special_tokens=True)
# score = extract_score_from_response(response[0])
# print(score)

In [24]:
import numpy as np
from scipy.stats import pearsonr
from tqdm import tqdm


model.eval()
predicted_scores = []
true_scores = []

for sample in tqdm(test_dataset):
  src = sample["src"]
  mt = sample["mt"]
  true_score = float(sample["zmean"])
  prompt = inference_prompt_style.format(src=src, mt=mt, score = "")
  inputs = tokenizer(prompt + tokenizer.eos_token, return_tensors="pt").to(model.device)

  with torch.no_grad():
    outputs = model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=1200,
        eos_token_id=tokenizer.eos_token_id,
        use_cache = True)
  response = tokenizer.batch_decode(outputs, skip_special_tokens=True)
  predicted_score = extract_score_from_response(response[0])
  predicted_scores.append(predicted_score)
  true_scores.append(true_score)

pearson_corr, _ = pearsonr(true_scores, predicted_scores)

100%|██████████| 125/125 [14:12<00:00,  6.82s/it]


In [25]:
print(pearson_corr)

0.023705307678204585


In [26]:
print(len(prompt))

762


In [27]:
print(type(prompt))

<class 'str'>


In [28]:
print(prompt[:50])

You are a multilingual translation evaluation expe


In [29]:
print(prompt)

You are a multilingual translation evaluation expert. Your task is to predict the quality score for translation pairs.
The quality score is an exact number between -2.0 and 2.0 and could fall into the following categories:
Scoring Criteria:
-2.0 to -1.0: Poor or very poor translation with meaning deviation or severe errors.
-1.0 to 0.0: Flawed translation but understandable.
0.0 to 1.0: Good translation in general.
1.0 to 2.0: Excellent or flawless translation.
The relative position of the scores in this range indicates subtle differences in translation quality.

### Source sentence:
ඇය ඔහුට දන්වා එවූයේ සිය සැමියා වූ රජු, නිභුරුරියා මියගොස් ඇති බවයි.

### MT sentence:
She informed him that the king of her husband, the native had died.

### Your score:



In [32]:
new_train_set = Dataset.from_pandas(traindf)
new_train_setje = new_train_set.map(
    formatting_prompts_func(),
    batched=True,
)



TypeError: formatting_prompts_func() got an unexpected keyword argument 'src'

In [31]:
print(new_train_setje['text'][16])

You are a multilingual translation evaluation expert. Your task is to predict the quality score for translation pairs.
The quality score is a number between -2.0 and 2.0 and could fall into the following categories:
Scoring Criteria:
-2.0 to -1.0: Poor or very poor translation with meaning deviation or severe errors.
-1.0 to 0.0: Flawed translation but understandable.
0.0 to 1.0: Good translation in general.
1.0 to 2.0: Excellent or flawless translation.
The relative position of the scores in this range indicates subtle differences in translation quality.

### Source sentence:
Confronting Scalpel for his misdeeds, she tortures him with his own scalpel, and leaves him disfigured and near death.

### MT sentence:
面对福音的恶行 ， 她用他自己的手术刀折磨他 ， 使他毁容 ， 差点就死了。

### Quality score:
-0.4941



In [34]:
def remove_gold_score(text):
    # 删除从 "### Quality score:" 到文本结尾的内容
    return re.sub(r"(### Quality score:\s*)(-?\d+(\.\d+)?)(.*)", r"\1", text)
stripped_prompts = [remove_gold_score(t) for t in train_dataset["text"]]

In [35]:
print(stripped_prompts[16])

You are a multilingual translation evaluation expert. Your task is to predict the quality score for translation pairs.
The quality score is a number between -2.0 and 2.0 and could fall into the following categories:
Scoring Criteria:
-2.0 to -1.0: Poor or very poor translation with meaning deviation or severe errors.
-1.0 to 0.0: Flawed translation but understandable.
0.0 to 1.0: Good translation in general.
1.0 to 2.0: Excellent or flawless translation.
The relative position of the scores in this range indicates subtle differences in translation quality.

### Source sentence:
Confronting Scalpel for his misdeeds, she tortures him with his own scalpel, and leaves him disfigured and near death.

### MT sentence:
面对福音的恶行 ， 她用他自己的手术刀折磨他 ， 使他毁容 ， 差点就死了。

### Quality score:




In [36]:
print(type(stripped_prompts))

<class 'list'>


In [37]:
print(len(stripped_prompts))

1000


In [42]:
print(len(new_train_set))
print(new_train_set[16])

1000
{'index': 1874, 'src': 'Confronting Scalpel for his misdeeds, she tortures him with his own scalpel, and leaves him disfigured and near death.', 'mt': '面对福音的恶行 ， 她用他自己的手术刀折磨他 ， 使他毁容 ， 差点就死了。', 'zmean': -0.4941006247433139, 'source_file': 'en-zh.tsv', 'root_folder': 'wmt22_fixed', 'file_type': 'zmean', 'global_index': 55874}


In [ ]:
model.eval()
predicted_scores = []
true_scores = []

for prompt, sample in tqdm(zip(stripped_prompts, new_train_set), total=len(new_train_set)):
    true_score = float(sample["zmean"])
    inputs = tokenizer(prompt + tokenizer.eos_token, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs.input_ids,
            attention_mask=inputs.attention_mask,
            max_new_tokens=1200,
            eos_token_id=tokenizer.eos_token_id,
            use_cache=True
        )

    response = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    predicted_score = extract_score_from_response(response[0])

    predicted_scores.append(predicted_score)
    true_scores.append(true_score)

# 评估
from scipy.stats import pearsonr
pearson_corr, _ = pearsonr(true_scores, predicted_scores)
print(f"Pearson correlation on training set: {pearson_corr:.4f}")

 29%|██▉       | 291/1000 [44:45<4:07:51, 20.98s/it]